# Recommending ICPSR datasets

Exploring the relationships between datasets based on their available variables.

## Import Modules

In [1]:
import os, json, csv
import pandas as pd

## Loading the data

In [2]:
studies_metadata_json = '../input_metadata/ICPSR_studies_metadata.json' # 23MB
vars_metadata_csv = '../input_metadata/all_vars.csv'                    # 1.3GB
doi_xwalk_csv = '../input_metadata/doi_xwalk.csv'                       # 509KB

### Functions

In [12]:
def get_study_url(var_url):
    '''Get url to dataset for var'''
    study_url= var_url.split('/datasets/')[0].replace('ssvd/', '')
    return study_url


def xwalk_dict(xwalk_csv):
    '''Load xwalk csv into a native dictionary for faster lookup'''
    xwalk_lookup = {}
    with open( xwalk_csv, 'r', encoding='utf-8' ) as ifile:
        reader = csv.reader(ifile)
        header = next(reader)
        for row in reader:
            xwalk_lookup[row[0]] = row[1]
    return xwalk_lookup

def trim_dataset_name(dataset_name):
    try:
        dataset_name = dataset_name.replace("Taken from: ", "").strip()
    except AttributeError as ae:
        print("{}: {}".format(ae, dataset_name))
    return dataset_name

In [4]:
vars_df = pd.read_csv(vars_metadata_csv)

In [13]:
vars_df.head()

,var_id,var_name,var_url,var_label,var_dataset,study_url
0,1,DRG7ING4,https://www.icpsr.umich.edu/icpsrweb/ICPSR/ssv...,Ingredient code #4 for medication #7,Taken from: National Hospital Ambulatory Medic...,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...
1,2,DRG7ING5,https://www.icpsr.umich.edu/icpsrweb/ICPSR/ssv...,Ingredient code #5 for medication #7,Taken from: National Hospital Ambulatory Medic...,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...
2,3,GEN8,https://www.icpsr.umich.edu/icpsrweb/ICPSR/ssv...,Generic name code for medication #8,Taken from: National Hospital Ambulatory Medic...,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...
3,4,PRESCR8,https://www.icpsr.umich.edu/icpsrweb/ICPSR/ssv...,Prescription status code for medication #8,Taken from: National Hospital Ambulatory Medic...,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...
4,5,CONTSUB8,https://www.icpsr.umich.edu/icpsrweb/ICPSR/ssv...,Controlled substance code for medication #8,Taken from: National Hospital Ambulatory Medic...,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...


In [6]:
len(vars_df)

4960317

In [7]:
len(vars_df.var_url.unique())

4960317

In [8]:
len(vars_df.var_name.unique())

1470415

In [9]:
vars_df['study_url'] = vars_df.apply(lambda x: get_study_url(x['var_url']), axis=1)

In [14]:
vars_df['dataset_name'] = vars_df.apply(lambda x: trim_dataset_name(x['var_dataset']), axis=1)

'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' object has no attribute 'replace': nan
'float' objec

In [15]:
vars_df.head()

,var_id,var_name,var_url,var_label,var_dataset,study_url,dataset_name
0,1,DRG7ING4,https://www.icpsr.umich.edu/icpsrweb/ICPSR/ssv...,Ingredient code #4 for medication #7,Taken from: National Hospital Ambulatory Medic...,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...,National Hospital Ambulatory Medical Care Surv...
1,2,DRG7ING5,https://www.icpsr.umich.edu/icpsrweb/ICPSR/ssv...,Ingredient code #5 for medication #7,Taken from: National Hospital Ambulatory Medic...,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...,National Hospital Ambulatory Medical Care Surv...
2,3,GEN8,https://www.icpsr.umich.edu/icpsrweb/ICPSR/ssv...,Generic name code for medication #8,Taken from: National Hospital Ambulatory Medic...,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...,National Hospital Ambulatory Medical Care Surv...
3,4,PRESCR8,https://www.icpsr.umich.edu/icpsrweb/ICPSR/ssv...,Prescription status code for medication #8,Taken from: National Hospital Ambulatory Medic...,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...,National Hospital Ambulatory Medical Care Surv...
4,5,CONTSUB8,https://www.icpsr.umich.edu/icpsrweb/ICPSR/ssv...,Controlled substance code for medication #8,Taken from: National Hospital Ambulatory Medic...,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...,National Hospital Ambulatory Medical Care Surv...


#### Load xwalk dictionary

In [16]:
xwalk_dict = xwalk_dict(doi_xwalk_csv)

In [17]:
xwalk_dict

{'https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04406': 'https://doi.org/10.3886/ICPSR04406.v2',
 'https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04409': 'https://doi.org/10.3886/ICPSR04409.v1',
 'https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04411': 'https://doi.org/10.3886/ICPSR04411.v1',
 'https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04412': 'https://doi.org/10.3886/ICPSR04412.v3',
 'https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04414': 'https://doi.org/10.3886/ICPSR04414.v1',
 'https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04417': 'https://doi.org/10.3886/ICPSR04417.v1',
 'https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04418': 'https://doi.org/10.3886/ICPSR04418.v1',
 'https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04419': 'https://doi.org/10.3886/ICPSR04419.v2',
 'https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04420': 'https://doi.org/10.3886/ICPSR04420.v2',
 'https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04421': 'https://doi.org/10.3

In [18]:
list(vars_df.study_url.head())

['https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04406',
 'https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04406',
 'https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04406',
 'https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04406',
 'https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04406']

## Explore the data

Rough exploration of variables related to datasets (studies)

In [19]:
datasets_df = pd.read_json(studies_metadata_json)

In [20]:
datasets_df.head()

,coverages,date,description,identifier,source,subjects,title
0,"[United States, 1824--1968]",1999-04-26,<p>Please read the <hi>collection notes</hi> b...,http://doi.org/10.3886/ICPSR00001.v3,official election returns provided by state go...,"[candidates, congressional elections, counties...","United States Historical Election Returns, 182..."
1,"[United States, 1788--1990]",1995-06-05,This data collection provides the names of can...,http://doi.org/10.3886/ICPSR00002.v5,official election returns provided by state go...,"[candidates, constituencies, elections, politi...","Candidate Name and Constituency Totals, 1788-1990"
2,"[United States, 1790--1970]",2005-12-22,Detailed county and state-level ecological or ...,http://doi.org/10.3886/ICPSR00003.v1,(1) United States Bureau of the Census decenni...,"[census data, counties, demographic characteri...","Historical, Demographic, Economic, and Social ..."
3,[1789--1998],2010-05-06,Roll call voting records for both chambers of ...,http://doi.org/10.3886/ICPSR00004.v3,Materials for the House from the 1st through t...,"[eighteenth century, historical data, legislat...",United States Congressional Roll Call Voting R...
4,"[United States, 1969]",2005-12-22,This study contains national data prepared for...,http://doi.org/10.3886/ICPSR00005.v1,,"[congressional elections, demographic characte...","Data Confrontation Seminar, 1969: United State..."


## Approaches To Determining Similarity between datasets

1.**TF-IDF vectors**

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel #from https://stackoverflow.com/questions/12118720/python-tf-idf-cosine-to-find-document-similarity
import numpy as np

In [57]:
documents = vars_df[['study_url','dataset_name', 'var_name', 'var_label']]

In [58]:
documents.head()

,study_url,dataset_name,var_name,var_label
0,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...,National Hospital Ambulatory Medical Care Surv...,DRG7ING4,Ingredient code #4 for medication #7
1,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...,National Hospital Ambulatory Medical Care Surv...,DRG7ING5,Ingredient code #5 for medication #7
2,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...,National Hospital Ambulatory Medical Care Surv...,GEN8,Generic name code for medication #8
3,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...,National Hospital Ambulatory Medical Care Surv...,PRESCR8,Prescription status code for medication #8
4,https://www.icpsr.umich.edu/icpsrweb/ICPSR/stu...,National Hospital Ambulatory Medical Care Surv...,CONTSUB8,Controlled substance code for medication #8


We are only going to develop a vector space composed of dataset variable descriptions.

In [25]:
tfidf_vec = TfidfVectorizer()

In [35]:
descriptions = documents['var_label']

In [37]:
descriptions = descriptions.fillna('')

In [38]:
descriptions_list = list(descriptions)

In [39]:
tfidf_matrix = tfidf_vec.fit_transform(descriptions_list)

In [40]:
print(tfidf_matrix.shape)

(4960317, 469398)


2. **Select a random document by index**

In [59]:
DOCUMENT_TARGET = 4488
#documents.iloc[ DOCUMENT_TARGET , : ]

In [60]:
#sample = list(documents.iloc[ DOCUMENT_TARGET , : ])

In [61]:
documents_list = documents.values.tolist()
sample = documents_list[DOCUMENT_TARGET]

In [62]:
sample

['https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04420',
 'Census of Governments, 1992: Finance Statistics - Final Restructured County-Area Summaries File with Added Identifiers.',
 'V_E17',
 'V_E17: Amount in $1000']

In [63]:
cos_sim = linear_kernel(tfidf_matrix[DOCUMENT_TARGET], tfidf_matrix).flatten()

In [64]:
filt_vars = np.asarray([x for x in cos_sim.tolist() if x > 0.10 and x < 0.9999], dtype=np.float32)

In [65]:
related_vars = cos_sim.argsort()[:-25:-1]

In [66]:
related_vars

array([   4488,    5009,    3748,    4784,    7611,    4291,    3495,
       1795941, 2641412,  751756, 1296662, 1202064, 4171631, 1393440,
       4171227, 4172048, 1394261, 1393844, 1202054,    4125,    4135,
          4127,    4124,    4128])

In [67]:
for i in related_vars:
    print(cos_sim[i])

1.0
1.0
1.0
0.585984875568094
0.585984875568094
0.585984875568094
0.585984875568094
0.3946482335580719
0.3828642195978902
0.3828642195978902
0.3828642195978902
0.36224350174046954
0.36124276478126466
0.36124276478126466
0.36124276478126466
0.36124276478126466
0.36124276478126466
0.36124276478126466
0.35589173106268185
0.3519254425917707
0.3519254425917707
0.3519254425917707
0.3519254425917707
0.3519254425917707


In [68]:
print("Most similar to: {}".format(documents_list[DOCUMENT_TARGET]))
column_id = documents_list[DOCUMENT_TARGET][0]
for i in related_vars:
    # remove any results where the variable is in the same dataset
    # and remove any scores that are above 0.90 (as they are probably the same variable)
    if cos_sim[i] < 0.90 and column_id != documents_list[i][0]:
        print("score: {} \n Document: {}".format(cos_sim[i], documents_list[i]))

Most similar to: ['https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04420', 'Census of Governments, 1992: Finance Statistics - Final Restructured County-Area Summaries File with Added Identifiers.', 'V_E17', 'V_E17: Amount in $1000']
score: 0.585984875568094 
 Document: ['https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/04423', 'Census of Governments, 1997: Finance Statistics - County Area Data File.', 'AMOUNT', 'Amount (in $1000)']
score: 0.3946482335580719 
 Document: ['https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/08678', 'ANES 1986 Time Series Study - Bias Data File.', 'V863072', 'POP IN 1000 S']
score: 0.3828642195978902 
 Document: ['https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/02407', 'American National Election Studies, 1992-1997: Combined File.', 'V923066', 'POPULATION IN 1000']
score: 0.3828642195978902 
 Document: ['https://www.icpsr.umich.edu/icpsrweb/ICPSR/studies/00064', 'English County Borough Data, 1951-1967  .', 'V2', "POPULATION IN 1000'S"]
score: 0.382